In [15]:
import torch

In [16]:
from model import GptLanguageModel
from common import GptConfig

In [17]:
hyperparameters = GptConfig()

In [18]:
model = GptLanguageModel(hyperparameters)
model.load_state_dict(torch.load('model_weights.pth'))
m = model.to(model.device)

In [19]:
tokenizer = m.tokenizer

In [20]:
start_str = "\n"
encoded_input = tokenizer.encode(start_str)
idx = torch.tensor(encoded_input, dtype=torch.long, device=model.device).unsqueeze(0)
encoded_input

[1, 29871, 13]

In [21]:
print(tokenizer.decode(m.generate(idx = idx, max_new_tokens=model.block_size-len(encoded_input))[0].tolist()))

<s> 
 It was so she would tell sure it around her mom. Susinkathers and asked her the vendor." C away and Jane laughed.

One day, M}(ile of the beach, her mum said, "Hello at it to the day. But then, the ground, let's full funny of her. It was and she didn't believe the bottom of herself so fast. She finally would even sprayed around.

 initi and wished she noticed that it was the skyroom. She couldn'dle the first. Justed his rare mom came out, pigeon for a couch. 3atboard with the story again and tough voice.

Jack was so happy, her mom pointed down and spots, hugged polit home. Her mom nodded to play together, feeling backyard she looked at the flightter became able to the plate pictures with the other eyes of it them cray, and use her insideedly!

Her mom showed sat over the phone and enjoyed it. But it was so learned everyone, whying all you to explore â€œQuat up is delicious with you.". And cleaning!â€™s important to play with it
